In [1]:
# Importing the libraries

import torch as th
import torch.nn as nn
from torchtext import vocab
import pickle as pl
from tqdm import tqdm
import numpy as np



# Import model
from Models.RNN import GRUNet



C:\Users\Nikolaj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load Data
with open('data/English_encodings.pkl', 'rb') as f:
    english_encodings,english_sentences,Paddings_en,Vocab_en = pl.load(f)
with open('data/French_encodings.pkl', 'rb') as f:
    french_encodings,french_sentences,Paddings_fr,Vocab_fr = pl.load(f)

# Get the vocabulary size
src_vocab_size = Vocab_fr.__len__()
tgt_vocab_size = Vocab_en.__len__()
src_padding_idx = Vocab_fr.__getitem__('<Pad>')
tgt_padding_idx = Vocab_en.__getitem__('<Pad>')


# Load Train,Vaildation and Test data
with open('data/Train_data.pkl', 'rb') as f:
    X_train,Y_train,src_padding_mask,tgt_padding_mask = pl.load(f)

with open('data/Validation_data.pkl', 'rb') as f:
    X_vali,Y_vali,src_padding_mask_vali = pl.load(f)

with open('data/Test_data.pkl', 'rb') as f:
    X_test,Y_test,src_padding_mask_test = pl.load(f)

tgt_mask = th.triu(th.full((27, 27), float('-inf')), diagonal=1)

In [3]:
batch_size = 50
n_train = 10000
n_batches = int(n_train/batch_size)

# Define Training Function
def train(model, optimizer, loss_fn, n_epochs, n_batches, X_train_batches, Y_train_batches, src_padding_idx):

    predict_token = th.zeros((n_batches,1))
    predict_token[:] = Vocab_fr.__getitem__('<Pad>')

    for epoch in range(n_epochs):
        loss_epoch = 0
        for batch in range(n_batches):
            # Get the data
            X_batch = X_train_batches[batch]
            Y_batch = Y_train_batches[batch]

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            y_pred = model(X_batch,predict_token)

            # Compute the loss
            loss = 0 
            for sample in range(batch_size):
                loss += loss_fn(y_pred[sample], Y_batch[sample])

            # Backward pass
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Print the loss
            loss_epoch += loss.item()

        print('Epoch: %d, Batch: %d, Loss: %f' % (epoch, batch, loss_epoch/n_train))

In [4]:
d_model = 512
dim_input = d_model
dim_output = d_model
dim_recurrent = d_model
num_sequence = 27
output = tgt_vocab_size
src_padding_idx = Vocab_en.__getitem__('<Pad>')

Model = GRUNet(dim_input, dim_recurrent, num_sequence, dim_output, src_padding_idx,src_vocab_size,tgt_vocab_size)

# Define the loss function
loss_fn = nn.CrossEntropyLoss()

# Define the optimizer LSTM
optimizer = th.optim.Adam(Model.parameters(), lr=0.001)

#define the number of epochs
n_epochs = 1000

# Define the number of batches
n_batches = 20

#train(Model, optimizer, loss_fn, n_epochs, n_batches, X_train, Y_train, src_padding_idx)


# Model(X_train[0])


C:\Users\Nikolaj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\nn\modules\rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [5]:

predict_token = th.tensor([Vocab_en.__getitem__('<Start>')],dtype = th.int32)

predict_token_batch = th.zeros((n_batches,1),dtype=th.int32)
predict_token_batch[:] = Vocab_fr.__getitem__('<Pad>')


out = Model(X_test[0],predict_token)
out.shape

RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 1-D tensor

In [ ]:
test_tensor = th.ones((10,5,2))
test_tensor[:][0] = th.tensor([1,2])
test_tensor

tensor([[[1., 2.],
         [1., 2.],
         [1., 2.],
         [1., 2.],
         [1., 2.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]]])

In [ ]:
Y_test[0]

tensor([206,  31,  13,  12,  14,   0,   6,  84,   1,   7,  30,  13,  12,   0,
          6,  87,   2, 204, 205, 205, 205, 205, 205, 205, 205, 205, 205])